In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.data.dtypes

user_id            int32
signup_month       int32
treat              int32
pre_spends       float64
post_spends      float64
age              float64
gender          category
industry        category
dtype: object

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

    p-value  statistic   pass
B  0.755673   0.311177  False
p-value      float64
statistic    float64
pass            bool
dtype: object
          p-value
TTest┴┴  0.755673
p-value
pass
    p-value  statistic   pass
B  0.727866     0.0138  False
p-value      float64
statistic    float64
pass            bool
dtype: object
           p-value
KSTest┴┴  0.727866
p-value
pass


c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [5]:
result.analysis_tables

{'GroupSizes┴┴':                      B
 control size    5000.0
 test size       5000.0
 control size %    50.0
 test size %       50.0,
 'GroupDifference┴┴':                                    B
 post_spends control mean  452.287178
 post_spends test mean     452.041933
 post_spends difference     -0.245244
 post_spends difference %   -0.054223,
 'TTest┴┴':           p-value  statistic   pass
 TTest┴┴  0.755673   0.311177  False,
 'KSTest┴┴':            p-value  statistic   pass
 KSTest┴┴  0.727866     0.0138  False,
 'OneAAStatAnalyzer┴┴':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0       0.755673         0.0        0.727866          0.0         0.737135}

In [6]:
AADictReporter().report(result)

{'random_state': None,
 'B post_spends control mean': 452.2871777777778,
 'B post_spends test mean': 452.0419333333333,
 'B post_spends difference': -0.245244444444495,
 'B post_spends difference %': -0.05422316981202879,
 'B control size': 5000.0,
 'B test size': 5000.0,
 'B control size %': 50.0,
 'B test size %': 50.0,
 'TTest p-value': 0.7556729102509778,
 'TTest pass': 0.0,
 'KSTest p-value': 0.7278658490549254,
 'KSTest pass': 0.0,
 'mean test score': 0.7371348694536094}